In [1]:
import tensorflow as tf

# TensorFlow 2.0 Notes

Link to [TensorFlow Guide](https://www.tensorflow.org/guide/effective_tf2).

### Effective TensorFlow 2

- Look for useful tools in `tf.keras.metrics` and `tf.keras.optimizers`.
- Keep track of your variables! If you lose track of a `tf.Variable`, it gets garbage collected.
- In TensorFlow 2.0, you can decorate a Python function using `tf.function()` to mark it for JIT compilation so that TensorFlow runs it as a single graph
- To help users avoid having to rewrite their code when adding `@tf.function`, AutoGraph converts a subset of Python constructs into their TensorFlow equivalents:

#### Recommendations for Idiomatic TensorFlow 2.0

- Refactor your code into smaller functions __it's not necessary to decorate each of these smaller functions with `tf.function`__; only use tf.function to decorate high-level computations - for example, one step of training or the forward pass of your model.
- Use Keras layers and models to manage variables (pretty much identical to the PyTorch API
- Combine `tf.data.Datasets` and `@tf.function`. Dataset is the best way to stream memory from disk, wrapping code in @tf.function takes advantage of dataset asynchonous loading

- `tf.metrics` aggregates data and `tf.summary` logs them.Metrics are stateful: They accumulate values and return a cumulative result when you call .result(). Clear accumulated values with `.reset_states()`.

- se `tf.config.experimental_run_functions_eagerly()` when debugging


### Better performance with tf.function and AutoGraph
 At the center of this merger is `tf.function`, which allows you to transform a subset of Python syntax into portable, high-performance TensorFlow graphs.
 
 When you annotate a function with tf.function, you can still call it like any other function. But it will be compiled into a graph, which means you get the benefits of faster execution, running on GPU or TPU, or exporting to SavedModel.

In [ ]:
@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

AutoGraph supports common Python statements like while, for, if, break, continue and return, with support for nesting.

n real applications batching is essential for performance. __The best code to convert to AutoGraph is code where the control flow is decided at the batch level.__ If making decisions at the individual example level, try to use batch APIs to maintain performance.

In [ ]:
def square_if_positive_vectorized(x):
    return tf.where(x > 0, x ** 2, x)

square_if_positive_vectorized(tf.range(-5, 5))

tf.function can give you significant speedup over eager execution, at the cost of a slower first-time execution. This is because when executed for the first time, the function is also traced into a TensorFlow graph.bb

### Keras


In Keras, you assemble layers to build models. A model is (usually) a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model.

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

After the model is constructed, configure its learning process by calling the `compile` method:

- Loss functions are specified by name or by passing a callable object from the tf.keras.losses module.
- metrics: Used to monitor training. These are string names or callables from the tf.keras.metrics module.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Use the Datasets API to scale to large datasets or multi-device training. Pass a `tf.data.Dataset` instance to the fit method:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

A callback is an object passed to a model to customize and extend its behavior during training. You can write your own custom callback, or use the built-in tf.keras.callbacks that include:

- `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at regular intervals.
- `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning rate.
- `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation performance has stopped improving.
- `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using TensorBoard.

In [ ]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# this requires a model with the same architecture.
model.load_weights('./weights/my_model')

__Keras Functional API__

In [ ]:
inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')
model.summary()

https://www.tensorflow.org/guide/keras/train_and_evaluate

https://www.tensorflow.org/guide/keras/save_and_serialize

https://www.tensorflow.org/guide/estimator

https://www.tensorflow.org/guide/eager

https://www.tensorflow.org/guide/variable

https://www.tensorflow.org/guide/tensor

https://www.tensorflow.org/guide/data

https://www.tensorflow.org/guide/checkpoint

https://www.tensorflow.org/guide/distributed_training

https://www.tensorflow.org/guide/gpu